In [5]:
import pandas as pd 

In [6]:
df = pd.read_csv('example/descriptors.csv')

/tmp/ipykernel_55840/422331916.py:1: DtypeWarning: Columns (92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('example/descriptors.csv')


In [11]:

to_drop = [
    "function_id",
    "graph_id",
    "edge_source",
    "edge_dest",
    "is_causal",
]

rocs = {}

df_copy = df.copy()
df_copy['d2c_prediction'] = 0
feat_importances = []

for function_id in df["function_id"].unique():
    testing_set = df[df["function_id"] == function_id]
    training_set = df[df["function_id"] != function_id]

    X_train = training_set.drop(columns=to_drop)
    y_train = training_set["is_causal"]

    X_test = testing_set.drop(columns=to_drop)

    y_test = testing_set["is_causal"]

    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier(n_estimators=100, random_state=42,n_jobs=50)

    clf.fit(X_train, y_train)

    f_imp = clf.feature_importances_
    f_imp = pd.DataFrame(f_imp, index=X_train.columns, columns=["importance"])
    f_imp = f_imp.sort_values(by="importance", ascending=False)
    f_imp["function_id"] = function_id
    print(f_imp.head(20).index, f_imp.head(20).importance.sum())
    feat_importances.append(f_imp.head(20).index)

    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)[:, 1]

    df_copy.loc[df_copy["function_id"] == function_id, "d2c_prediction"] = y_pred_proba
    df_copy.loc[df_copy["function_id"] == function_id, "d2c_is_causal"] = y_pred

    from sklearn.metrics import roc_auc_score

    rocs[function_id] = roc_auc_score(y_test, y_pred_proba)

Index(['coeff_cause', 'kurtosis_ef', 'kurtosis_ca', 'HOC_1_3', 'coeff_eff',
       'HOC_3_1', 'eff_m_cau_q1', 'eff_m_cau_q0', 'eff_m_cau_q6',
       'eff_m_cau_q5', 'skewness_ca', 'skewness_ef', 'eff_m_cau_q3',
       'eff_m_cau_q2', 'm_eff_q5', 'm_eff_q0', 'eff_m_cau_q4', 'HOC_2_1',
       'm_eff_q1', 'HOC_1_2'],
      dtype='object') 0.5831137140900178
Index(['coeff_cause', 'kurtosis_ef', 'kurtosis_ca', 'HOC_1_3', 'coeff_eff',
       'HOC_3_1', 'eff_m_cau_q0', 'eff_m_cau_q1', 'skewness_ef', 'skewness_ca',
       'eff_m_cau_q3', 'eff_m_cau_q5', 'eff_m_cau_q6', 'eff_m_cau_q2',
       'm_eff_q5', 'm_eff_q1', 'HOC_2_1', 'm_eff_q6', 'eff_m_cau_q4',
       'HOC_1_2'],
      dtype='object') 0.5827971571088302
Index(['coeff_cause', 'kurtosis_ef', 'kurtosis_ca', 'HOC_1_3', 'coeff_eff',
       'HOC_3_1', 'eff_m_cau_q1', 'eff_m_cau_q6', 'skewness_ef', 'skewness_ca',
       'eff_m_cau_q0', 'eff_m_cau_q5', 'eff_m_cau_q3', 'm_eff_q5',
       'eff_m_cau_q2', 'eff_m_cau_q4', 'HOC_2_1', 'HOC_1_2', 'm

KeyboardInterrupt: 

In [4]:
feat_importances[importance]

,importance,function_id
coeff_cause,0.104039,linear
kurtosis_ef,0.053909,linear
kurtosis_ca,0.044576,linear
HOC_1_3,0.035064,linear
coeff_eff,0.033019,linear
...,...,...
mca_mca_cau_q6,0.000000,17
mca_mca_cau_q5,0.000000,17
mca_mca_cau_q4,0.000000,17
mca_mca_cau_q3,0.000000,17


In [14]:
rocs

{'linear': 0.9889913098727224,
 'sigmoid': 0.8136220166442343,
 'interaction': 0.9870191550515789,
 'poly_deg_1_2': 0.9798721874583944,
 'poly_deg_2_3': 0.8948899147904122,
 'nonlinear_<built-in function sin>': 0.9676447272135945,
 'nonlinear_<built-in function sqrt>': 0.986228611259644}

In [15]:
cols = ['function_id', 'graph_id', 'edge_source', 'edge_dest', 'd2c_is_causal']
df_copy[cols].to_csv('example/d2c_predictions.csv', index=False)